In [1]:
#import libraries
import pandas as pd
import numpy as np
import csv
import ast
import re
from prettytable import PrettyTable

#spelling correction
import enchant
from spellchecker import SpellChecker
from autocorrect import Speller
from textblob import TextBlob


#data visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.io as pio
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator

#NLP & ML libraries
from nltk import FreqDist
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
#import cleaned data

def list_converter(text):
    #to revert list->str conversion from pd.read_csv
    return ast.literal_eval(text)


data = pd.read_csv('Data/training_corpus.csv', converters ={'tokens':list_converter})

In [18]:
data

,index,text_type,ID,year,long_text,clean_text,word_count,tokens
0,0,comment,gtfo2hl,2021,"*Cuntry roads, take me hoem*",cuntry roads hoem,3,"[cuntry, road, hoem]"
1,1,comment,gtfqkbv,2021,"That’s been there for several years, sent a pi...",years sent pic cuntry friend long time ago,8,"[year, send, pic, cuntry, friend, long, time, ..."
2,2,comment,gtfou07,2021,I am single and I have not traveled to any cun...,single traveled cuntry past year,5,"[single, travel, cuntry, past, year]"
3,3,comment,gtfrgpe,2021,What happens when you shop at dragon mart...,happens shop dragon mart,4,"[happen, shop, dragon, mart]"
4,4,comment,gthiiwi,2021,"That’s just absolutely hilarious, is this in t...",absolutely hilarious springs souk,4,"[absolutely, hilarious, spring, souk]"
...,...,...,...,...,...,...,...,...
99181,100367,submission,14f46ji,2023,"Best beauty saloons in Dubai? Hello fellas, I ...",best beauty saloons dubai hello fellas moved w...,35,"[good, beauty, saloon, dubai, hello, fellas, m..."
99182,100368,submission,14f4uyi,2023,Found the r/dubai redditors who kept telling m...,found r dubai redditors kept telling know navi...,10,"[find, r, dubai, redditor, keep, tell, know, n..."
99183,100369,submission,14f4ri3,2023,Scam ? Healthy.line My sister has a CBD debit ...,scam healthy line sister cbd debit card month ...,47,"[scam, healthy, line, sister, cbd, debit, card..."
99184,100370,submission,14f4k3r,2023,Thoughts on Expo City properties? Anyone else ...,thoughts expo city properties checked expo cit...,21,"[thought, expo, city, property, check, expo, c..."


## **Bag of Words Model** (from Sklearn)

In [20]:
#lemmatization was done on tokens, not clean text

In [19]:
#create CountVectorizer object
cv = CountVectorizer()

#convert corpus to matrix of word vectors
#bow_matrix = vectorizer.fit_transform(data)

#create vocabulary with indices for each word
#cv_vectorizer = vectorizer.fit(data['clean_text'].values)

#vocabulary = vectorizer.vocabulary_
#print unique words and their indices
#print ("vocabulary: ", vocabulary.items()[:5])

#create word vectors for each word
bow_matrix = cv.fit_transform(data['clean_text'].values)

#print matrix as 2-d vector arrays
#print (bow_matrix.toarray())

bow_matrix


<99186x47566 sparse matrix of type '<class 'numpy.int64'>'
	with 1262927 stored elements in Compressed Sparse Row format>